In [ ]:
import pandas as pd
#import io
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.preprocessing import MinMaxScaler
import numpy as np
#%matplotlib inline
import seaborn as sns
#import matplotlib.pyplot as plt

import plotly.graph_objs as go 
#from plotly.offline import plot

import plotly.express as px

from IPython.display import display_html
# color pallette
cnf, dth, rec, act,wth,sth = '#393e46', '#ff2e63', '#21bf73', '#fe9801','#456fe3','#78ffee'

air_pollution =pd.read_csv('long-run-air-pollution.csv')


In [ ]:
air_pollution.head()

In [ ]:
air_pollution.drop(['Code'], axis=1, inplace=True)

In [ ]:
air_pollution.info()

In [ ]:
air_pollution.shape

In [ ]:
air_pollution.describe()

In [ ]:
def Missing (X):
    total = X.isnull().sum().sort_values(ascending=False)
    percent = round(X.isnull().sum().sort_values(ascending=False)/len(X)*100, 2)
    missing = pd.concat([total, percent], axis = 1, keys = ['Total', 'Percent'])
    return( missing)
#plots number of dataframes side by side
def SideSide(*args):
    html_str=''
    for df in args:
        html_str+=df.to_html()
    display_html(html_str.replace('table','table style="display:inline"'),raw = True)

In [ ]:
# Fill empty values with NaN
air_pollution = air_pollution.fillna(np.nan)
#finds missing values
missing_air_pollution = Missing(air_pollution)
print('country year DATA')
SideSide(missing_air_pollution)
print('\n\n  MISSING  DATA ')

In [ ]:
def max_polluted_country(pollutant):
    x1 = air_pollution[[pollutant,'Entity']].groupby(["Entity"]).mean().sort_values(by=pollutant,ascending=False).reset_index()
    x1[pollutant] = round(x1[pollutant],2)
    return x1[:10].style.background_gradient(cmap='OrRd')

In [ ]:
from IPython.display import display_html
def display_side_by_side(*args):
    html_str=''
    for df in args:
        html_str+=df.render()
    display_html(html_str.replace('table','table style="display:inline"'),raw=True)

In [ ]:
nox = max_polluted_country('Nitrogen oxide (NOx)')
so2 = max_polluted_country('Sulphur dioxide (SO₂) emissions')
co = max_polluted_country('Carbon monoxide (CO) emissions')
nh3 = max_polluted_country('Ammonia (NH₃) emissions')
bc = max_polluted_country('Black carbon (BC) emissions')
nmvoc = max_polluted_country('Non-methane volatile organic compounds (NMVOC) emissions')


display_side_by_side(nox, so2, co, nh3, bc, nmvoc )

In [ ]:
data = air_pollution.copy()

In [ ]:

# Create traces for all countries and global data
fig = go.Figure()

# Add global data traces
global_data = data.groupby('Year').sum().reset_index()
fig.add_trace(go.Scatter(x=global_data['Year'], y=global_data['Nitrogen oxide (NOx)'],
                         mode='lines', name='Global Nitrogen oxide (NOx)', visible=True, hovertemplate='NOx: %{y}<extra></extra>'))
fig.add_trace(go.Scatter(x=global_data['Year'], y=global_data['Sulphur dioxide (SO₂) emissions'],
                         mode='lines', name='Global Sulphur dioxide (SO₂)', visible=True,  hovertemplate='SO₂: %{y}<extra></extra>'))
fig.add_trace(go.Scatter(x=global_data['Year'], y=global_data['Carbon monoxide (CO) emissions'],
                         mode='lines', name='Global Carbon monoxide (CO)', visible=True,  hovertemplate='CO: %{y}<extra></extra>'))
fig.add_trace(go.Scatter(x=global_data['Year'], y=global_data['Black carbon (BC) emissions'],
                         mode='lines', name='Global Black carbon (BC)', visible=True,  hovertemplate='BC: %{y}<extra></extra>'))
fig.add_trace(go.Scatter(x=global_data['Year'], y=global_data['Ammonia (NH₃) emissions'],
                         mode='lines', name='Global Ammonia (NH₃)', visible=True,  hovertemplate='NH₃: %{y}<extra></extra>'))
fig.add_trace(go.Scatter(x=global_data['Year'], y=global_data['Non-methane volatile organic compounds (NMVOC) emissions'],
                         mode='lines', name='Global Non-methane volatile <br>organic compounds (NMVOC)', visible=True,  hovertemplate='NMVOC: %{y}<extra></extra>'))

# Add country-specific traces
countries = data['Entity'].unique()
for country in countries:
    country_data = data[data['Entity'] == country]
    fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data['Nitrogen oxide (NOx)'],
                             mode='lines', name=f'{country} Nitrogen oxide (NOx)', visible=False,  hovertemplate='NOx: %{y}<extra></extra>'))
    fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data['Sulphur dioxide (SO₂) emissions'],
                             mode='lines', name=f'{country} Sulphur dioxide (SO₂)', visible=False,  hovertemplate='SO₂: %{y}<extra></extra>'))
    fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data['Carbon monoxide (CO) emissions'],
                             mode='lines', name=f'{country} Carbon monoxide (CO)', visible=False,  hovertemplate='CO: %{y}<extra></extra>'))
    fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data['Black carbon (BC) emissions'],
                             mode='lines', name=f'{country} Black carbon (BC)', visible=False,  hovertemplate='BC: %{y}<extra></extra>'))
    fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data['Ammonia (NH₃) emissions'],
                             mode='lines', name=f'{country} Ammonia (NH₃)', visible=False))
    fig.add_trace(go.Scatter(x=country_data['Year'], y=country_data['Non-methane volatile organic compounds (NMVOC) emissions'],
                             mode='lines', name=f'{country} Non-methane volatile <br> organic compounds (NMVOC)', visible=False, hovertemplate='NH₃: %{y}<extra></extra>'))

# Get the minimum and maximum years
min_year = data['Year'].min()
max_year = data['Year'].max()

# Create buttons for each country and global data
buttons = []
buttons.append(dict(
    method='update',
    label='Global',
    args=[{'visible': [True] * 6 + [False] * len(countries) * 6},
          {'title': 'Global Emissions Over Time', 'yaxis.title.text': 'Emissions (tonnes)'}]
))

for country in countries:
    visibility = [False] * 6  # Hide global data
    visibility += [trace.name.startswith(f'{country}') for trace in fig.data[6:]]
    buttons.append(dict(
        method='update',
        label=country,
        args=[{'visible': visibility},
              {'title': f'Emissions of Air pollutants in {country} Over Time ({min_year} to {max_year})', 'yaxis.title.text': 'Emissions (tonnes)'}]
    ))

# Update layout for better interactivity and add buttons
fig.update_layout(
     title=f'Global Emissions of Air Pollutants Over Time ({min_year} to {max_year})',
    xaxis=dict(
        title='Year',
        range=[min_year, max_year]
    ),
    yaxis_title='Emissions (tonnes)',  # Updated y-axis title to reflect the correct units
    hovermode='x unified',  # This enables a vertical line on hover with unified tooltips
    legend_title_text='Emissions',
    template='plotly_white',
    updatemenus=[{
        'buttons': buttons,
        'direction': 'down',
        'showactive': True,
        'x': 1.35,
        'xanchor': 'right',
        'y': 1.15,
        'yanchor': 'top',
        'pad': {'r': 10, 't': 10},
        'active': 0,
        'bgcolor': 'lightgray',
        'bordercolor': 'gray',
        'font': {'size': 12},
        'type': 'dropdown'
    }],
    annotations=[
        dict(text='Select Country or Region', x=1.35, xanchor='right', y=1.25, yanchor='top', showarrow=False)
    ]
)

# Show the figure
fig.show()


In [ ]:
air_pollution['Year'] = pd.to_datetime(air_pollution['Year'], format='%Y')

In [ ]:
air_pollution.set_index('Year', inplace=True)

In [ ]:
scaler = MinMaxScaler()
numeric_cols = air_pollution.select_dtypes(include=[np.number]).columns
air_pollution[numeric_cols] = scaler.fit_transform(air_pollution[numeric_cols])

In [ ]:
# Calculate the correlation matrix
correlation_matrix = air_pollution.corr()

# Create a heatmap
plt.figure(figsize=(10,8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm')
plt.show()

In [ ]:
correlation_with_co_emissions = correlation_matrix['Carbon monoxide (CO) emissions'].sort_values(ascending=False)
print(correlation_with_co_emissions)

In [ ]:
air_pollution.columns


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.statespace.sarimax import SARIMAX
import numpy as np


# Prepare the data for global CO emissions
world_data = data[data['Entity'] == 'World']
world_data.set_index('Year', inplace=True)
co_emissions = world_data['Carbon monoxide (CO) emissions']

# Train the ARIMA model
model = SARIMAX(co_emissions, order=(1, 1, 1), seasonal_order=(0, 0, 0, 0))
results = model.fit()

# Forecast the next 10 years
forecast = results.get_forecast(steps=10)
forecast_index = np.arange(co_emissions.index[-1] + 1, co_emissions.index[-1] + 11)
forecast_values = forecast.predicted_mean
forecast_conf_int = forecast.conf_int()

# Create a DataFrame for the forecasted CO emissions
forecast_df = pd.DataFrame({
    'Year': forecast_index,
    'Carbon monoxide (CO) emissions': forecast_values,
    'Lower CI': forecast_conf_int.iloc[:, 0],
    'Upper CI': forecast_conf_int.iloc[:, 1]
})

# Merge the historical and forecasted CO emissions data
historical_co_emissions = co_emissions.reset_index()
historical_co_emissions.columns = ['Year', 'Carbon monoxide (CO) emissions']
merged_df = pd.concat([historical_co_emissions, forecast_df], ignore_index=True)

# Create traces for the global data
fig = go.Figure()

# Historical CO Emissions
fig.add_trace(go.Scatter(
    x=historical_co_emissions['Year'],
    y=historical_co_emissions['Carbon monoxide (CO) emissions'],
    mode='lines',
    name='Historical CO Emissions',
    line=dict(color='blue'),
    hovertemplate='Historical: %{y}<extra></extra>' 
))

# Forecasted CO Emissions
fig.add_trace(go.Scatter(
    x=forecast_df['Year'],
    y=forecast_df['Carbon monoxide (CO) emissions'],
    mode='lines',
    name='Forecasted CO Emissions',
    line=dict(dash='dash', color='red'),
    hovertemplate='Forecasted: %{y}<extra></extra>'
    
))

# Lower Confidence Interval
fig.add_trace(go.Scatter(
    x=forecast_df['Year'],
    y=forecast_df['Lower CI'],
    mode='lines',
    line=dict(color='lightgrey'),
    showlegend=False,
    hoverinfo='skip'
))

# Upper Confidence Interval
fig.add_trace(go.Scatter(
    x=forecast_df['Year'],
    y=forecast_df['Upper CI'],
    fill='tonexty',
    mode='lines',
    line=dict(color='lightgrey'),
    fillcolor='rgba(68, 68, 68, 0.3)',
    showlegend=False,
    hoverinfo='skip'
))

# Update layout for better interactivity
fig.update_layout(
    title='Global CO Emissions Over Time with Forecast (2023 to 2032)',
    xaxis=dict(
        title='Year',
        range=[1750, 2032],  # Set the desired range for the x-axis
        tickvals=np.arange(1750, 2033, 10)
    ),
    yaxis=dict(title='CO Emissions (tonnes)'),
    hovermode='x unified',
    legend_title_text='Emissions',
    template='plotly_white'
)
# Save the figure as an HTML file
fig.write_html("co_forecast.html")
# Show the figure
fig.show()
